In [1]:
import awkward as ak 
import numpy as np
import matplotlib.pyplot as plt
import fastjet
from coffea.nanoevents import NanoEventsFactory, EDM4HEPSchema
import dask_awkward as dak
import hist.dask as hda
import uproot
from ak_tools import ak_equals
import pickle
import matplotlib as mpl
import os
import tensorflow as tf

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '3' # set GPU
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [3]:
events = NanoEventsFactory.from_root( 
    {"/data/linear/Pe2e2hh.eL.pR.n000.d_dstm_15806_0_patched_collections_edm4hep_test_Jim.root"
    :"events"},
    schemaclass=EDM4HEPSchema,
    permit_dask=True,
    metadata = {'b_field':5},
).events()

/usr/local/lib/python3.8/dist-packages/cppyy_backend/loader.py:139: UserWarning: No precompiled header available (/usr/local/lib/python3.8/dist-packages/cppyy_backend/etc not writable, set CLING_STANDARD_PCH); this may impact performance.
  warnings.warn('No precompiled header available (%s); this may impact performance.' % msg)
/home/jbrewster/.local/lib/python3.8/site-packages/numba/core/cpu.py:99: UserWarning: Numba extension module 'cppyy.numba_ext' failed to load due to 'ModuleNotFoundError(No module named 'llvmlite.llvmpy')'.
  numba.core.entrypoints.init_all()
/home/jbrewster/projects/coffea/src/coffea/nanoevents/factory.py:63: UserWarning: Skipping PARAMETERS as it is it cannot be interpreted by Uproot
  warnings.warn(
/home/jbrewster/projects/coffea/src/coffea/nanoevents/factory.py:45: UserWarning: Skipping _intMap as it is not interpretable by Uproot
  warnings.warn(
/home/jbrewster/projects/coffea/src/coffea/nanoevents/factory.py:52: UserWarning: Skipping _intMap.first as it

In [4]:
# set up needed arrays
file = uproot.open("/data/linear/Pe2e2hh.eL.pR.n000.d_dstm_15806_0_patched_collections_edm4hep_test_Jim.root")

evs = file['events']
uproot_parinds = (evs[f'_MCParticlesSkimmed_parents/_MCParticlesSkimmed_parents.index']).array()
uproot_daughinds = (evs[f'_MCParticlesSkimmed_daughters/_MCParticlesSkimmed_daughters.index']).array() 

pshape = (events.MCParticlesSkimmed.parents_end - events.MCParticlesSkimmed.parents_begin).compute()
par_inds = ak.unflatten(uproot_parinds,ak.flatten(pshape),axis=1)

dshape = (events.MCParticlesSkimmed.daughters_end - events.MCParticlesSkimmed.daughters_begin).compute()
daugh_inds = ak.unflatten(uproot_daughinds,ak.flatten(dshape),axis=1)

pdgids = events.MCParticlesSkimmed.pdgId.compute()


arr_reco = events.RecoMCTruthLink.reco_index.compute()
arr_mc = events.RecoMCTruthLink.mc_index.compute()
sort_reco = arr_reco[ak.argsort(arr_reco)]
sort_mc = arr_mc[ak.argsort(arr_reco)]

proper_indices = ak.unflatten(sort_mc,ak.flatten(ak.run_lengths(sort_reco),axis=1),axis=1)

In [5]:
pfos = events.PandoraPFOs.compute()
mc = events.MCParticlesSkimmed.compute()

In [6]:
def jet_truehiggs(reco_particles,mc_particles,true_higgs,reco_mc_index,jet_constits_index,condition='pR',pcrit=(50,5)):
    '''
    compares jet constituents to true higgs products 
    
    Parameters:
    ----------
    reco_particles :: array
        array with all the reco particles
    
    mc_particles :: array
        array with all the mc particles
    
    true_higgs :: array 
        array containing the mc indices of all the children of the higgs in each event 
    
    reco_mc_index :: array 
        array containing a list of mc indices for each reco index
    
    jet_constits_index :: array 
        indices of the pfos contained in the jets in the reco particle array
    
    condition :: str (optional)
        'pR' - the link with the closest momentum match is from the true higgs, to within 50% of the reco momentum 
               or 5 GeV, otherwise the closest \Delta R between the two closest momentum matches
               (criteria can be changed with pcrit)
        'p' - the link with the closest momentum match is from the true higgs
        'any' - any of one of the links from reco particles to mc particles is from the true higgs 
        'all' - all of the links from reco particles to mc particles is from the true higgs 
    
    pcrit :: tuple (optional)
        tuple defining the criteria for using momentum when condition = 'pR' as 
        (percent,absolute) FINISH DESCRIPTION
        
    Returns:
    -------
    
    '''
    matrix = False
    
    # throw error if any of the things are not the right lengths
    if not (len(reco_particles) == len(mc_particles) and len(mc_particles) == len(true_higgs) and len(true_higgs) == len(reco_mc_index) and len(reco_mc_index) == len(jet_constits_index)):
        raise ValueError('first 5 arrays must have the same number of events')
    
    jet_mcindices = reco_mc_index[jet_constits_index]
    jet_con_in_higgs = []
    
    if condition == 'pR':
        # complete (right now this is the same as 'p')
        matched_gen = ak.unflatten(mc_particles[ak.flatten(reco_mc_index,axis=2)],ak.flatten(ak.num(reco_mc_index,axis=2)),axis=1)
        
        mc_reco_cartesian = ak.argcartesian({'mc':matched_gen.p,'reco':ak.singletons(reco_particles.p,axis=1)},axis=2)
        mc_p_argmin = ak.singletons(ak.argmin(abs(matched_gen.p[mc_reco_cartesian['mc']] - ak.singletons(reco_particles.p,axis=1)[mc_reco_cartesian['reco']]),axis=2),axis=1)
        
        all_jet_inds = ak.flatten(reco_mc_index[mc_p_argmin],axis=2)[jet_constits_index]
        
        for n in range(len(all_jet_inds)):
            jet_con_in_higgs.append(np.isin(all_jet_inds[n],true_higgs[n]))

        jet_con_in_higgs = ak.Array(jet_con_in_higgs)
        
    elif condition == 'any' or condition == 'all':
        all_jet_inds = ak.flatten(jet_mcindices,axis=2)
        
        jet_con_in_higgs = []

        for n in range(len(all_jet_inds)):
            jet_con_in_higgs.append(np.isin(all_jet_inds[n],true_higgs[n]))

        jet_con_in_higgs = ak.Array(jet_con_in_higgs)
        
        if condition == 'any':
            jet_con_in_higgs = ak.any(ak.unflatten(jet_con_in_higgs,ak.flatten(ak.num(jet_mcindices,axis=2)),axis=1),axis=2)
        else:
            jet_con_in_higgs = ak.all(ak.unflatten(jet_con_in_higgs,ak.flatten(ak.num(jet_mcindices,axis=2)),axis=1),axis=2)
        
    elif condition == 'p':
        matched_gen = ak.unflatten(mc_particles[ak.flatten(reco_mc_index,axis=2)],ak.flatten(ak.num(reco_mc_index,axis=2)),axis=1)
        
        mc_reco_cartesian = ak.argcartesian({'mc':matched_gen.p,'reco':ak.singletons(reco_particles.p,axis=1)},axis=2)
        mc_p_argmin = ak.singletons(ak.argmin(abs(matched_gen.p[mc_reco_cartesian['mc']] - ak.singletons(reco_particles.p,axis=1)[mc_reco_cartesian['reco']]),axis=2),axis=1)
        
        all_jet_inds = ak.flatten(reco_mc_index[mc_p_argmin],axis=2)[jet_constits_index]
                
        if matrix:
            jet_con_in_higgs = ak_equals(all_jet_inds,true_higgs)
        else:
            for n in range(len(all_jet_inds)):
                jet_con_in_higgs.append(np.isin(all_jet_inds[n],true_higgs[n]))

            jet_con_in_higgs = ak.Array(jet_con_in_higgs)
        
    else:
        raise ValueError('condition must be \'p\', \'any\', or \'all\'')
    
    
    return jet_con_in_higgs

In [7]:
pickle_path = '/fast_scratch_1/jbrewster/dihiggs_ML/storing_misc/'

higgs_children_0 = pickle.load(open(pickle_path + 'higgs_children_0.pickle','rb'))
higgs_children_1 = pickle.load(open(pickle_path + 'higgs_children_1.pickle','rb'))
higgs_daughter_tree_arr_0 = pickle.load(open(pickle_path + 'higgs_daughter_tree_arr_0.pickle','rb'))
higgs_daughter_tree_arr_1 = pickle.load(open(pickle_path + 'higgs_daughter_tree_arr_1.pickle','rb'))

In [8]:
mupair = dak.combinations(events.PandoraPFOs[abs(events.PandoraPFOs.pdgId) == 13], 2, fields=["mu1", "mu2"])
pairmass = (mupair.mu1 + mupair.mu2).mass
muonsevent = dak.any(
    (pairmass > 80)
    & (pairmass < 100)
    & (mupair.mu1.charge == -mupair.mu2.charge),
    axis=1,
)
muonsevent_c = muonsevent.compute()

jetdef = fastjet.JetDefinition(fastjet.kt_algorithm,1)

pfopair = dak.argcombinations(events.PandoraPFOs, 2, fields=["p1", "p2"])

all_muons_mask = (abs(events.PandoraPFOs[pfopair.p1].pdgId) == 13) & (abs(events.PandoraPFOs[pfopair.p2].pdgId) == 13)

invmass = (events.PandoraPFOs[pfopair.p1][all_muons_mask] + events.PandoraPFOs[pfopair.p2][all_muons_mask]).mass

inds = dak.singletons(dak.argmin(abs(invmass - 91.2), axis=1))


mu1ind = pfopair.p1[all_muons_mask][inds]
mu2ind = pfopair.p2[all_muons_mask][inds]

In [ ]:
m1 = mu1ind[muonsevent].compute()
m2 = mu2ind[muonsevent].compute()

p = events.PandoraPFOs[muonsevent].compute()

local_inds = ak.local_index(p)
total_mask = ((ak_equals(local_inds, m1)) | (ak_equals(local_inds, m2))) != True

In [ ]:
pfos_h0 = jet_truehiggs(pfos[muonsevent_c][total_mask],
                        mc[muonsevent_c],
                        higgs_children_0[muonsevent_c],
                        proper_indices[muonsevent_c][total_mask],
                        ak.local_index(pfos[muonsevent_c][total_mask]),
                        'p')

pfos_h1 = jet_truehiggs(pfos[muonsevent_c][total_mask],
                        mc[muonsevent_c],
                        higgs_children_1[muonsevent_c],
                        proper_indices[muonsevent_c][total_mask],
                        ak.local_index(pfos[muonsevent_c][total_mask]),
                        'p')

In [ ]:
# making truth array 
true_arr = ak.fill_none(
    ak.mask(
        ak.fill_none(
            ak.mask(
                ak.fill_none(
                    ak.mask(pfos_h0,pfos_h0 != True),ak.Array([1,0,0])),
                pfos_h1 != True
            ),
            ak.Array([0,1,0])
        ),
        pfos_h0 | pfos_h1,
    ),
    ak.Array([0,0,1]),
)

padded_true_arr = ak.fill_none(ak.pad_none(true_arr,np.max(ak.num(true_arr,axis=1))),ak.Array([-1,-1,-1]))

padded_true_np = np.array(ak.unflatten(ak.unflatten(ak.ravel(padded_true_arr),ak.num(padded_true_arr)*3),3,axis=1))

In [ ]:
# flattening pfo dict into tuples 
# possibly tell it the data type here 

masked_pfos = pfos[muonsevent_c][total_mask]

# masked_pfos_tupled = ak.zip([masked_pfos.x,masked_pfos.y,masked_pfos.z,masked_pfos.E])

# masked_pfos_tupled = ak.zip([np.log(masked_pfos.p)*masked_pfos.x/masked_pfos.p,
#                              np.log(masked_pfos.p)*masked_pfos.y/masked_pfos.p,
#                              np.log(masked_pfos.p)*masked_pfos.z/masked_pfos.p,
#                              np.log(masked_pfos.E)])

masked_pfos_tupled = ak.zip([np.log(masked_pfos.p)*masked_pfos.x/masked_pfos.p,
                             np.log(masked_pfos.p)*masked_pfos.y/masked_pfos.p,
                             np.log(masked_pfos.p)*masked_pfos.z/masked_pfos.p,
                             (np.log(masked_pfos.E)-np.mean(np.log(masked_pfos.E)))/np.max(np.log(masked_pfos.E)-np.mean(np.log(masked_pfos.E)))])

# masked_pfos_tupled = ak.zip([masked_pfos.x,masked_pfos.y,masked_pfos.z,np.log(masked_pfos.E)])

padded_pfo_arr = ak.fill_none(ak.pad_none(masked_pfos_tupled,np.max(ak.num(pfos,axis=1))),ak.Array([-1,-1,-1,-1]))

padded_pfo_np = np.array(ak.unflatten(ak.unflatten(ak.ravel(padded_pfo_arr),ak.num(padded_pfo_arr)*4),4,axis=1))

In [ ]:
np.max(np.log(masked_pfos.p)*masked_pfos.x/masked_pfos.p)

In [20]:
np.max(np.log(masked_pfos.E)-np.mean(np.log(masked_pfos.E)))

6.5207872

In [51]:
train_top = int(len(padded_pfo_arr)*0.7)
val_top = int(train_top + len(padded_pfo_arr)*0.2)

In [52]:
train_top

83722

In [53]:
train_data = padded_pfo_np[:train_top]
train_truth = padded_true_np[:train_top]

test_data = padded_pfo_np[train_top:val_top]
test_truth = padded_true_np[train_top:val_top]

val_data = padded_pfo_np[val_top:]
val_truth = padded_true_np[val_top:]

In [54]:
data_path = '/fast_scratch_1/jbrewster/dihiggs_ML/'

In [55]:
max_points_file = data_path + 'max_points.txt'
file = open(max_points_file, 'w')
file.write(str(ak.num(train_data)[0]))
file.close()

In [56]:
# save with X and Y labels
# np.savez(data_path + 'train/dihiggs',X=train_data,Y=train_truth)
# np.savez(data_path + 'test/dihiggs',X=test_data,Y=test_truth)
# np.savez(data_path + 'val/dihiggs',X=val_data,Y=val_truth)

In [57]:
for n in range(int(np.ceil(len(train_data)/6000))):
    top = (n+1)*6000
    if top < len(train_data):
        np.savez(data_path + 'train/dihiggs' + str(n),X=train_data[n*6000:top],Y=train_truth[n*6000:top])
    else:
        np.savez(data_path + 'train/dihiggs' + str(n),X=train_data[top:],Y=train_truth[top:])

In [58]:
for n in range(int(np.ceil(len(test_data)/6000))):
    top = (n+1)*6000
    if top < len(test_data):
        np.savez(data_path + 'test/dihiggs' + str(n),X=test_data[n*6000:top],Y=test_truth[n*6000:top])
    else:
        np.savez(data_path + 'test/dihiggs' + str(n),X=test_data[top:],Y=test_truth[top:])

In [59]:
for n in range(int(np.ceil(len(val_data)/6000))):
    top = (n+1)*6000
    if top < len(val_data):
        np.savez(data_path + 'val/dihiggs' + str(n),X=val_data[n*6000:top],Y=val_truth[n*6000:top])
    else:
        np.savez(data_path + 'val/dihiggs' + str(n),X=val_data[top:],Y=val_truth[top:])